In [15]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import regex as re
from multiprocessing import Pool
import sys

In [16]:
base_url = "https://www.kegg.jp/kegg-bin/uniprot_list?ko=" # append KO group to end of url
def transform_int2knumber(knint):
    knstr = str(knint)
    while len(knstr) < 5:
        knstr = "0" + knstr
    return "K" + knstr
not_kog_ids = []
with open("not_knumber.log", "r") as f:
    lines = f.readlines()
    for line in lines:
        knumber = line.strip()
        not_kog_ids.append(knumber)
kog_ids_raw = [transform_int2knumber(k) for k in range(1, 27110)]
kog_ids = [k for k in kog_ids_raw if k not in not_kog_ids]

In [148]:
def extract_uid_htmlrow(rowtext):
    uid = re.findall(r'">(.*)<\/a', rowtext)[0]
    return uid
def extract_descriptor_htmlrow(rowtext, altformat = False):
    if altformat:
        descriptor = re.findall(r'[^\n].*$', descriptor_row)[0].split("\xa0")[-1]
        return descriptor
    descriptor = re.findall(r';\s(.*)', rowtext)[0] 
    return descriptor

In [168]:
def get_uids(kogid): # also gets descriptor for KOG
    uids = []
    url = base_url + kogid
    data = requests.get(url)
    html = BeautifulSoup(data.text, 'html.parser')
    try: 
        descriptor_row = html.div.text
    except:
        with open("not_knumber_anymore.log", "a") as outfile:
            print(kogid, file=outfile)
            return
    print("completed request for {}".format(kogid))
    try:
        descriptor = extract_descriptor_htmlrow(descriptor_row)
    except: # row is formated without ";" separating the gene alias and descriptor
        descriptor = extract_descriptor_htmlrow(descriptor_row, True)
    print("got descriptor")

    all_tds = html.find_all("td")
    for i in range(2, len(html.find_all("td")), 4):
        rowtext = str(all_tds[i])
        uid = extract_uid_htmlrow(rowtext)
        uids.append(uid)

    output = "data/kog_uid/{}.tsv".format(kogid)
    with open(output, "a") as outfile:
        for uid in uids:
            print("\t".join([kogid, uid, descriptor]), file=outfile)
#get_uids("K15032")

In [ ]:
section, num_sections = sys.argv[1], sys.argv[2]

In [7]:
for kog_id in kog_ids[:8]:
    get_uids(kog_id)

completed request for K00001
completed request for K00002
completed request for K00003
completed request for K00004
completed request for K00005
completed request for K00006
completed request for K00007
completed request for K00008


In [10]:
n = 1094
section = 2
sections = 8

for i in range(0, 1094, int(1094/8)):
    print(i)


0
136
272
408
544
680
816
952
1088


In [71]:
kog_ids_100 = kog_ids

In [85]:
num_sections = 10
section = 10

In [86]:
len(kog_ids_100) // num_sections

2644

In [87]:
section_indices = [i for i in range(0, len(kog_ids_100), len(kog_ids_100) // num_sections)]
start_range = section_indices[section-1]
if section == num_sections:
    end_range = len(kog_ids_100)
else:
    end_range = section_indices[section]

In [95]:
start_range, end_range

(23796, 26442)

In [97]:
kog_ids_100[len(kog_ids_100)]

IndexError: list index out of range

In [94]:
len(kog_ids_100)

26442

In [99]:
a[len(a)]

IndexError: list index out of range